In [1]:
%%bash

# Dependencies
pip install docx2txt
pip install gensim
pip install keras
pip install nltk
pip install -U scikit-learn
pip install python-docx
pip install tensorflow
pip install pandas
pip install nltk
pip install spacy
pip install csv
python3 -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv
2023-07-28 20:24:03.852864: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 20:24:05.114510: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 20:24:05.120396: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 20:24:08.415433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-28 20:24:14.083510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devi

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# Use if en_core_web_sm not installable via python3 in terminal:
# import spacy
# spacy.cli.download("en_core_web_sm")

In [31]:
import pandas as pd

file_path = "Bloom and SOLO Verbs.xlsx"

xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

number_of_sheets = 4
blooms_levels = 6

remembering = []
understanding = []
applying = []
analysing = []
evaluating = []
creating = []


for i in range(0, number_of_sheets):
    bloom = pd.read_excel(xls, sheet_names[i])
    for j in range(blooms_levels):
        current_column = bloom.columns[j]
        column_values = bloom[current_column].values.tolist()
        match current_column:
            case "Remembering":
                remembering = set(list(column_values) + list(remembering))
            case "Understanding":
                understanding = set(list(column_values) + list(understanding))
            case "Applying":
                applying = set(list(column_values) + list(applying))
            case "Analysing":
                analysing = set(list(column_values) + list(analysing))
            case "Evaluating":
                evaluating = set(list(column_values) + list(evaluating))
            case "Creating":
                creating = set(list(column_values) + list(creating))
                

        

print(remembering)
print(understanding)
print(applying)
print(analysing)
print(evaluating)
print(creating)


{'Match', 'Name', 'identify', 'read', 'Recall', 'duplicate', 'Describe', 'indicate', 'cite', 'recall', 'underline', 'Tell', 'define', 'repeat', 'name', 'Define', 'draw', 'select', 'tell', 'Write', 'label', 'recite', 'describe', 'List', 'Identify', 'reproduce', 'Draw', 'match', 'write', 'review', 'tabulate', 'Find', 'memorise', 'Label', 'copy', nan, 'Arrange', 'order', 'relate', 'Quote', 'outline', 'show', 'list', 'quote', 'record', 'Choose', 'locate', 'Recite', 'recognise', 'state'}
{'Demonstrate', 'identify', 'convert', 'characterise', 'Predict', 'assosciate', 'cite', 'indicate', 'infer', 'interpret', 'estimate', 'compare', 'interpolate', 'express', 'defend', 'Report', 'paraphrase', 'rewrite', 'generalise', 'select', 'tell', 'comprehend', 'classify', 'predict', 'clarify', 'demonstrate', 'describe', 'distinguish', 'Identify', 'extrapolate', 'extend', 'match', 'review', 'explain', 'articulate', 'discuss', 'report', 'observe', nan, 'Conclude', 'summarise', 'organise', 'Illustrate', 'Exem

In [1]:
## This code loads the vector file into the word_vectors variable
## Download the vector file from https://fasttext.cc/docs/en/english-vectors.html (first file on the website), unzip the file and store in your local development folder
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

from gensim.models import KeyedVectors

# Path to the downloaded .vec file
path_to_vectors = 'wiki-news-300d-1M.vec'
# path_to_vectors = 'wiki.en.vec'
# Load the word vectors
word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# Find similar words
similar_words = word_vectors.most_similar('cat')

# Calculate word similarity
similarity = word_vectors.similarity('cat', 'dog')

# Perform vector arithmetic
result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']

In [111]:
import spacy

## Function to identify verbs in a sentence
def identify_verbs(sentence):
    # Load the English language model in spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    
    return verbs

In [112]:
# Extract verbs and corresponding Bloom level from csv file
import csv

path_to_bloom_verbs = 'Bloom_Verbs.csv'
bloom_verbs = []

# Extract Verbs and corresponding Bloom level from csv file
import csv
with open(path_to_bloom_verbs, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0] != '':
            bloom_verbs.append(row) 

bloom_levels = [
    ["Remembering"],
    ["Understanding"],
    ["Applying"],
    ["Analysing"],
    ["Evaluating"],
    ["Creating"],
]

# Store verbs into array at corresponding level
for i in range(1, len(bloom_verbs)):
    if bloom_verbs[i][1] != 0:
        match bloom_verbs[i][1]:
            case "Remembering":
                bloom_levels[0].append(str.lower(bloom_verbs[i][0]))
            case "Understanding":
                bloom_levels[1].append(str.lower(bloom_verbs[i][0]))
            case "Applying":
                bloom_levels[2].append(str.lower(bloom_verbs[i][0]))
            case "Analysing":
                bloom_levels[3].append(str.lower(bloom_verbs[i][0]))
            case "Evaluating":
                bloom_levels[4].append(str.lower(bloom_verbs[i][0]))
            case "Creating":
                bloom_levels[5].append(str.lower(bloom_verbs[i][0]))

In [135]:
# Extract LOs and corresponding mapped Bloom level from csv file
path_to_mappings = 'eng4701-mappings.csv'
sentences = []

import csv
with open(path_to_mappings, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0] != '':
            sentences.append(row[0]) 

In [157]:
# ## Main piece of code that performs the mapping

# import math
# import pprint
# # wiki word vectors no uppercase
# # TODO: Modify this section of the code to use the bloom level verbs from Arragon's spreadsheet, will also need to modify

# output_bloom_levels = []

# from gensim.models import Word2Vec
# from gensim.test.utils import common_texts

# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("wordnet")

# stop_words = set(stopwords.words("english"))
# lemmatizer = WordNetLemmatizer()

# # Array of all the PLOs and ULOs (We can couple them together as we're trying to identify Bloom/Solo level here)
# lo_sentence_array = []

# # TODO: train CLO classification with all data instead of just one course.
# for sentence in sentences:
#     tokens = nltk.word_tokenize(sentence)
#     cleaned_tokens = [
#         lemmatizer.lemmatize(token)
#         for token in tokens
#         if token.lower() not in stop_words
#     ]
#     lo_sentence_array.append(cleaned_tokens)

# # build the vocabulary and train the model
# # IMPORTANT, N0TE THAT sg=1 flag specifies Word2Vec to use the Skip Gram Model as designated by the LSTM paper.
# model = Word2Vec(
#     sentences=lo_sentence_array, vector_size=100, window=5, min_count=1, workers=4, sg=1
# )

# # train the model with the course's ULOs and PLOs.
# model.train([tokens], total_examples=len([tokens]), epochs=10)

# lo_sentence_array
# failed_cases = []


# def bloom_mapping(sentences, final_bloom_level):
#     # identified_levels = []
#     final_level = None
#     passed_mappings = 0
#     failed_mappings = 0
#     total_mappings = len(sentences)
#     for i in range(1, len(sentences)): # Iterates over the LOs
#         verbs = identify_verbs(sentences[i])
#         # print("Verbs: ", verbs)
#         score = 0
#         verbs_similarity_score = []
#         for j in range(len(verbs)):     # Iterates over the verbs identified in each LO
#             bloom_verb_level_score = -1
#             for k in range(len(bloom_levels)):  # Iterates over Bloom levels
#                 bloom_level_score_final = -1
#                 for l in range(1, len(bloom_levels[k])):    # Iterates over verbs in each Bloom level
#                     similarity_score = -1   
#                     try:    # Currently some of the 'verbs' identified are phrases rather than words and it was throwing errors so this is a temp solution 
#                         similarity_score = word_vectors.similarity(verbs[j], bloom_levels[k][l])
#                     except:
#                         pass
#                     if similarity_score >= score:
#                         score = similarity_score
#                         final_level = bloom_levels[k][0]
#                         final_level_idx = k
#                         # print(bloom_levels[k][l] + " " + str(score))
#                     verb_score = {
#                         "verb": verbs[j],
#                         "score": similarity_score,
#                         "bloom_levels": bloom_levels[k][0]
#                     }
#                     verbs_similarity_score.append(verb_score)
#                 #     bloom_level_score_final += similarity_score
#                 # if bloom_level_score_final >= bloom_verb_level_score:
#                 #     bloom_verb_level_score = bloom_level_score_final
#                 #     final_level = bloom_levels[k][0]
#                 #     final_level_idx = k
#         if final_bloom_level[i].lower() == final_level.lower():
#             passed_mappings += 1
#         else:
#             failed_mappings += 1
#             # failed_cases.append((sentences[i], final_bloom_level[i], final_level, verbs))

#         output_bloom_levels.append(final_level_idx)
#         # print("Sentence: ", sentences[i])
#         # print("Identified blooms level: ", final_level, "\n")
#         sentence_data = {
#             "sentence": sentences[i],
#             "manually identified bloom level": final_bloom_level[i],
#             "automatically identified bloom level": final_level,
#             "verbs identified": verbs_similarity_score,
#         }
#         failed_cases.append(sentence_data)
# # print(output_bloom_levels)
#     # pprint(failed_cases[1])
#     mapping_percentage = math.ceil((passed_mappings/total_mappings)*100)
#     print("Total percentage of mappings passed: ", mapping_percentage, "%")
# ### Todos
# # Find a way to use skipgrams
# # This method only works for blooms since this paper is only based on blooms mapping

[nltk_data] Downloading package punkt to /home/musthafa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/musthafa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/musthafa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [162]:
## Main piece of code that performs the mapping - Approach 2 - 12% accuracy

import math
import pprint
# wiki word vectors no uppercase
# TODO: Modify this section of the code to use the bloom level verbs from Arragon's spreadsheet, will also need to modify
# Ideas for improving accuracy
#### Reduce the number of verbs.
#### Take more learning outcomes from the monash handbook website(need big dataset for this part) and identify verbs that are appearing multiple times 
#### The nummber of times that it appears could be set to a certain number ex: 5. 
#### If the verb doesnt appear atleast 5 times, we could remove the verb from our list of predefined verbs which will result in a shorter verb list

output_bloom_levels = []

from gensim.models import Word2Vec
from gensim.test.utils import common_texts

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Array of all the PLOs and ULOs (We can couple them together as we're trying to identify Bloom/Solo level here)
lo_sentence_array = []
bloom_levels_str = [
    "Remembering",
    "Understanding",
    "Applying",
    "Analysing",
    "Evaluating",
    "Creating",
]

# TODO: train CLO classification with all data instead of just one course.
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    cleaned_tokens = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token.lower() not in stop_words
    ]
    lo_sentence_array.append(cleaned_tokens)

# build the vocabulary and train the model
# IMPORTANT, N0TE THAT sg=1 flag specifies Word2Vec to use the Skip Gram Model as designated by the LSTM paper.
model = Word2Vec(
    sentences=lo_sentence_array, vector_size=100, window=5, min_count=1, workers=4, sg=1
)

# train the model with the course's ULOs and PLOs.
model.train([tokens], total_examples=len([tokens]), epochs=10)

lo_sentence_array
failed_cases = []


def bloom_mapping(sentences, final_bloom_level):
    # identified_levels = []
    final_level = None
    passed_mappings = 0
    failed_mappings = 0
    total_mappings = len(sentences)
    for i in range(1, len(sentences)): # Iterates over the LOs
        verbs = identify_verbs(sentences[i])
        # rem_sum = 0
        # und_sum = 0
        # app_sum = 0
        # ana_sum = 0
        # eval_sum = 0
        # cre_sum = 0
        score_list = [0, 0, 0, 0, 0, 0]
        for j in range(len(bloom_levels)):
            for k in range(1, len(bloom_levels[j])):
                similarity_score = 0
                for l in range(len(verbs)):
                    try:    # Currently some of the 'verbs' identified are phrases rather than words and it was throwing errors so this is a temp solution 
                        similarity_score += word_vectors.similarity(verbs[l], bloom_levels[j][k])
                    except:
                        pass
                match bloom_levels[j][0]:
                    case "Remembering":
                        score_list[0] = score_list[0] + similarity_score
                    case "Understanding":
                        score_list[1] = score_list[1] + similarity_score
                    case "Applying":
                        score_list[2] = score_list[2] + similarity_score
                    case "Analysing":
                        score_list[3] = score_list[3] + similarity_score
                    case "Evaluating":
                        score_list[4] = score_list[4] + similarity_score
                    case "Creating":
                        score_list[5] = score_list[5] + similarity_score
        max_score = 0
        max_score_id = None
        for m in range(len(score_list)):
            if max_score < score_list[m]:
                max_score = score_list[m]
                max_score_id = m

        final_level = bloom_levels_str[m]
        if final_bloom_level[i].lower() == final_level.lower():
            passed_mappings += 1
        else:
            failed_mappings += 1
            # failed_cases.append((sentences[i], final_bloom_level[i], final_level, verbs))

        # output_bloom_levels.append(final_level_idx)
        # print("Sentence: ", sentences[i])
        # print("Identified blooms level: ", final_level, "\n")
        sentence_data = {
            "sentence": sentences[i],
            "manually identified bloom level": final_bloom_level[i],
            "automatically identified bloom level": final_level,
            # "verbs identified": verbs_similarity_score,
        }
        failed_cases.append(sentence_data)
# print(output_bloom_levels)
    # pprint(failed_cases[1])
    mapping_percentage = math.ceil((passed_mappings/total_mappings)*100)
    print("Total percentage of mappings passed: ", mapping_percentage, "%")
### Todos
# Find a way to use skipgrams
# This method only works for blooms since this paper is only based on blooms mapping

[nltk_data] Downloading package punkt to /home/musthafa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/musthafa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/musthafa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [163]:
import csv

def extract_columns(csv_file, columns):
    extracted_data = {}
    
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the headers
        
        # Check if all specified columns exist in the CSV file
        for column in columns:
            if column not in headers:
                raise ValueError(f"Column '{column}' not found in the CSV file.")
        
        # Initialize separate arrays for each column
        for column in columns:
            extracted_data[column] = []
        
        # Extract data from specified columns
        for row in reader:
            for column in columns:
                column_index = headers.index(column)
                extracted_data[column].append(row[column_index])
    
    return extracted_data

# Example usage
csv_file = 'Learning outcomes manual mapping - Mappings - Testing.csv'
columns_to_extract = ['Learning outcomes', 'Final Bloom Level']
extracted_data = extract_columns(csv_file, columns_to_extract)
sentences = extracted_data['Learning outcomes']
final_bloom_level = extracted_data['Final Bloom Level']
bloom_mapping(sentences, final_bloom_level)



Total percentage of mappings passed:  0 %


In [155]:
# print(bloom_levels)
# problem - number of verbs in csv for each bloom level is not the same

# identified_verbs = failed_cases['verbs identified']
rem_sum = 0
und_sum = 0
app_sum = 0
ana_sum = 0
eval_sum = 0
cre_sum = 0
# for j in range(len(failed_cases)):
#     identified_verbs = failed_cases[j]['verbs identified']
#     for i in range(len(identified_verbs)):
#         bloomslevel = identified_verbs[i]['bloom_levels']
#         score = identified_verbs[i]['score']
#         match bloomslevel:
#             case "Remembering":
#                 rem_sum += score
#             case "Understanding":
#                 und_sum += score
#             case "Applying":
#                 app_sum += score
#             case "Analysing":
#                 ana_sum += score
#             case "Evaluating":
#                 eval_sum += score
#             case "Creating":
#                 cre_sum += score



# print(word_vectors.similarity("identify", "Remembering"))
# print(word_vectors.similarity("identify", "Understanding"))
# print(word_vectors.similarity("identify", "applying"))
# print(word_vectors.similarity("identify", "analysing"))
# print(word_vectors.similarity("identify", "evaluating"))
# print(word_vectors.similarity("identify", "Creating"))


print(bloom_levels)

for i in range(len(bloom_levels)):
    bloom_level = bloom_levels[i]
    print(len(bloom_level))
    for j in range(1, len(bloom_level)):
        final_score = 0
        try:
            similarity_score1 = word_vectors.similarity("identify", bloom_level[j])
            if (similarity_score1 >= 0.5):
                final_score += similarity_score1
            else:
                final_score = 0
            # similarity_score2 = word_vectors.similarity("include", bloom_level[j])
            # if (similarity_score2 >= 0.5):
            #     final_score += similarity_score2
        except:
            pass
        match bloom_level[0]:
            case "Remembering":
                rem_sum += final_score
            case "Understanding":
                und_sum += final_score
            case "Applying":
                app_sum += final_score
            case "Analysing":
                ana_sum += final_score
            case "Evaluating":
                eval_sum += final_score
            case "Creating":
                cre_sum += final_score

print(rem_sum)  
print(und_sum)
print(app_sum)
print(ana_sum)
print(eval_sum)
print(cre_sum)


[['Remembering', 'copy', 'define', 'draw', 'duplicate', 'find', 'label', 'list', 'memorise', 'name', 'quote', 'read', 'recall', 'recite', 'record', 'repeat', 'state', 'tabulate', 'underline'], ['Understanding', 'articulate', 'associate', 'clarify', 'comprehend', 'convert', 'discuss', 'exemplify', 'express', 'extend', 'extrapolate', 'give', 'give examples', 'interpolate', 'observe', 'paraphrase', 'report', 'represent', 'restate'], ['Applying', 'adapt', 'apply', 'back / back up', 'change', 'complete', 'compute', 'dramatise', 'employ', 'implement', 'interview', 'manipulate', 'operate', 'paint', 'simulate', 'sketch', 'use', 'utilise'], ['Analysing', 'analyse', 'break', 'break down', 'correlate', 'debate', 'deconstruct', 'deduce', 'detect', 'diagnose', 'diagram', 'divide', 'examine', 'figure', 'group', 'inspect', 'investigate', 'point out', 'research'], ['Evaluating', 'argue', 'assess', 'attach', 'convince', 'core', 'counsel', 'critique', 'decide', 'determine', 'evaluate', 'grade', 'judge',